# Modelagem preditiva para Análise de Crédito

## Introdução

O presente trabalho visa realizar uma modelagem preditiva, baseada nos pilares
da ciência de dados e do aprendizado de máquina. Para a análise, foi utilizada
uma versão modificada da base de dados
[default of credit card clients](https://github.com/TrainingByPackt/Data-Science-Projects-with-Python/raw/master/Data/default_of_credit_card_clients__courseware_version_1_21_19.xls)
da UCI, a qual contém cerca de 30000 contas de etiquetadas de maneira binária,
os quais indicam se o cliente conseguiu (ou não) pagar a fatura mínima de
crédito dentro dos ultimos 6 meses.

## O problema de negócios

O objetivo do presente trabalho se concentra na prática da modelagem preditiva,
bem como um melhor entendimento do problema central, *a análise de crédito*.

Segundo o [Sinfab/RS](https://sinbraf.com.br/voce-sabe-o-que-e-credito-2/), crédito é:

    CRÉDITO é uma fonte adicional de recursos que não são seus, mas obtidos de terceiros (bancos, financeiras, cooperativas de crédito e outros), que possibilita a antecipação do consumo para a aquisição de bens ou contratação de serviços. 

Nesse sentido, modelar crédito é medir o quão confiável o está a vida financeira
de uma conta. Se a conta apresentar indícios de que está passando por um período
menos estável, gostariamos de evitar liberar um crédito alto demais.

## Objetivos

Desenvolver um modelo preditivo capaz de classificar a probablilidade de uma conta bancária ficar inadimplente nos próximos mês, baseado nos dados históricos coletados.

## Dicionário de dados (Data Dictionary)

A documentação a respeito do conjunto de dados, e que vai ser nosso apoio durante
as etapas de limpeza, feature engineering e modelagem:

- *LIMIT_BAL*: representa o limite do cartão de crédito. (em dólares taiwaneses)
- *EDUCATION*: Grau de escolaridade (1=pós-graduação, 2=graduação, 3=ensino médio,
4=outro)
- *MARRIEGE*: Estado civil (1=casado, 2=solteiro, 3=outros)
- *AGE*: idade em anos
- *PAY_1-PAY_6*: registro de pagamentos, ordenados do mais recente para o mais antigo.
(em dólares taiwaneses)
- *BILL_AMT1-BILL_AMT6*: Estrato bancário, ordenados do mais recente para o mais antigo.
(em dólares taiwaneses)
- *default payment next month*: Nossa variável alvo, atrasou o pagamento? 1=sim, 0=não.

anotações extras que talvez possam ser importantes para a análise:

- Dados de 2005

# Obetenção dos dados

## Obtendo os dados via rede

para fins de praticidade, escrevi uma pequena automação para baixar os dados
nencessários do site da UCI e deixar em uma pasta a minha
disposição.

In [1]:
import os
import requests

def download_dataset(url, local_folder, file_name):
    if not os.path.exists(local_folder):
        os.makedirs(local_folder)

    response = requests.get(url)
    zip_file_path = os.path.join(local_folder, file_name)

    with open(zip_file_path, 'wb') as zip_file:
        zip_file.write(response.content)


In [2]:
download_dataset(
    url='https://github.com/TrainingByPackt/Data-Science-Projects-with-Python/raw/master/Data/default_of_credit_card_clients__courseware_version_1_21_19.xls',
    local_folder='datasets',
    file_name='credit_data.xls'
)

## Visualizando uma amostra aleatória

vamos acessaros dados e checar se está tudo funcionando como o esperado antes de
iniciarmos a nossa análise.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

credit_data = pd.read_excel(
    os.path.join('datasets', 'credit_data.xls'))

sample = credit_data.sample(5)
sample

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
25493,92a3ff40-a459,320000,2,1,2,29,0,0,0,0,...,34259,62546,106857,1955,2020,1605,30039,65000,5000,0
29237,2aaaad12-e02b,50000,1,2,2,23,0,0,0,0,...,49685,50252,9958,1785,2341,2128,344,359,800,0
10969,bc9c3c71-7c17,110000,2,3,1,42,0,0,0,0,...,119267,116142,113887,5836,5780,5000,4206,5000,5000,0
10374,c0f61d29-b97d,200000,2,2,2,26,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
2033,6ecd977c-1224,240000,2,1,1,27,1,-2,-1,-1,...,14350,14129,10222,0,402,14350,1129,1222,204,0


ficou um pouco difícil de excergar, o notebook omitiu alguns dados.

Vamos observar a amostra por partes:

In [4]:
sample.iloc[:, :10]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4
25493,92a3ff40-a459,320000,2,1,2,29,0,0,0,0
29237,2aaaad12-e02b,50000,1,2,2,23,0,0,0,0
10969,bc9c3c71-7c17,110000,2,3,1,42,0,0,0,0
10374,c0f61d29-b97d,200000,2,2,2,26,-2,-2,-2,-2
2033,6ecd977c-1224,240000,2,1,1,27,1,-2,-1,-1


In [5]:
sample.iloc[:, 10:20]

,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2
25493,0,0,42596,43548,33674,34259,62546,106857,1955,2020
29237,0,0,50511,50478,50146,49685,50252,9958,1785,2341
10969,0,0,116292,118781,121030,119267,116142,113887,5836,5780
10374,-2,-2,0,0,0,0,0,0,0,0
2033,0,0,0,0,402,14350,14129,10222,0,402


In [6]:
sample.iloc[:, 20:25]

,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
25493,1605,30039,65000,5000,0
29237,2128,344,359,800,0
10969,5000,4206,5000,5000,0
10374,0,0,0,0,0
2033,14350,1129,1222,204,0


No próximo tópico, vamos analisar o que cada feature representa.

no momento o que eu gostaria de fazer é mudar o nome da variável alvo. Além de
`default payment next month` ser um nome muito grande, está fora de padrão em
relação às outras variáveis.

In [7]:
credit_data.rename(columns={'default payment next month': 'has_defaulted'}, inplace=True)

# Conhecendo o conjunto de dados

Análise exploratória

Uma vez obtidos os dados, podemos iniciar nossa análise exploratória.

In [8]:
def print_shape(data) -> None:
    rows, columns = data.shape
    print(f'O conjunto possui {columns} colunas (features) e {rows} linhas.')

print_shape(credit_data)

O conjunto possui 25 colunas (features) e 30000 linhas.


### identificando os tipos de dados

In [9]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             30000 non-null  object
 1   LIMIT_BAL      30000 non-null  int64 
 2   SEX            30000 non-null  int64 
 3   EDUCATION      30000 non-null  int64 
 4   MARRIAGE       30000 non-null  int64 
 5   AGE            30000 non-null  int64 
 6   PAY_1          30000 non-null  object
 7   PAY_2          30000 non-null  int64 
 8   PAY_3          30000 non-null  int64 
 9   PAY_4          30000 non-null  int64 
 10  PAY_5          30000 non-null  int64 
 11  PAY_6          30000 non-null  int64 
 12  BILL_AMT1      30000 non-null  int64 
 13  BILL_AMT2      30000 non-null  int64 
 14  BILL_AMT3      30000 non-null  int64 
 15  BILL_AMT4      30000 non-null  int64 
 16  BILL_AMT5      30000 non-null  int64 
 17  BILL_AMT6      30000 non-null  int64 
 18  PAY_AMT1       30000 non-n

### Identificando os limites das variaveis numéricas

In [10]:
credit_data.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,has_defaulted
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,165760.989333,1.586500,1.833333,1.535900,35.108800,-0.132867,-0.164333,-0.219300,-0.263900,-0.288033,...,42791.362167,39884.398167,38480.350933,5613.321500,5.855410e+03,5174.387967,4776.089733,4754.749200,5164.223267,0.218733
std,130158.590432,0.513348,0.807699,0.542698,9.851592,1.191215,1.191096,1.162348,1.127519,1.144981,...,64090.316188,60606.644833,59406.836932,16539.094312,2.299256e+04,17565.538305,15532.893047,15239.070708,17712.664703,0.413394
min,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2034.000000,1534.000000,1080.000000,836.000000,7.217500e+02,371.000000,223.000000,170.750000,9.000000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,18759.500000,17835.500000,16643.000000,2084.500000,2.000000e+03,1776.000000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,53572.250000,49804.000000,48863.500000,5000.000000,5.000000e+03,4500.000000,4000.000000,4000.000000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.000000,621000.000000,426529.000000,528666.000000,1.000000


In [11]:
print(f'total de valores faltando: {credit_data.isnull().sum().sum()}')

total de valores faltando: 0


### Validando a integridade

In [12]:
credit_data.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'has_defaulted'],
      dtype='object')

In [13]:
credit_data['ID'].nunique()

29687

In [14]:
id_counts = credit_data['ID'].value_counts()
id_counts.value_counts()

count
1    29374
2      313
Name: count, dtype: int64

há uma diferença entre o número de numero de colunas e o número de Ids, o que
indica que há duplicações nos dados, dada a informação de que cada id deveria
ser único na tabela.

Criando uma mascara para os ids duplicados, é possivel selecioná-los para
visualizálos e, posteriormente, tratá-los adequaldamente.

In [15]:
duplicated_id_mask = id_counts == 2
duplicated_id_mask[0:3]

ID
ad23fe5c-7b09    True
1fb3e3e6-a68d    True
89f8f447-fca8    True
Name: count, dtype: bool

In [16]:
id_counts.index[0:3]

Index(['ad23fe5c-7b09', '1fb3e3e6-a68d', '89f8f447-fca8'], dtype='object', name='ID')

In [17]:
duplicated_ids = id_counts.index[duplicated_id_mask]
duplicated_ids = list(duplicated_ids)

len(duplicated_ids), duplicated_ids[0:3]

(313, ['ad23fe5c-7b09', '1fb3e3e6-a68d', '89f8f447-fca8'])

In [18]:
credit_data.loc[credit_data['ID'].isin(duplicated_ids[0:3]), :].head(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,has_defaulted
5033,89f8f447-fca8,320000,2,2,1,32,0,0,0,0,...,169371,172868,150827,8000,8000,5500,6100,6000,5000,0
5133,89f8f447-fca8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16727,1fb3e3e6-a68d,80000,1,2,2,33,2,2,0,0,...,27394,29922,31879,0,2000,2000,3000,2600,0,1
16827,1fb3e3e6-a68d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29685,ad23fe5c-7b09,50000,1,3,1,32,0,0,0,0,...,12882,8131,3983,3000,2871,1000,163,3983,3771,1
29785,ad23fe5c-7b09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


A conclusão que se pode chegar, visualizando os dados duplicados, é a de que,
de alguma forma, a query que originou o conjunto de dados produziu algumas linhas
com ID duplicado, mas com valores zerados.

A boa notícia é que não há problemática em remover essed dados.

In [19]:
df_zero_mask = credit_data == 0

In [20]:
feature_zero_mask = df_zero_mask.iloc[:, 1:].all(axis=1)

feature_zero_mask.sum()

315

In [21]:
df_clear_1 = credit_data.loc[~feature_zero_mask, :].copy()

df_clear_1.shape

(29685, 25)

Abaixo temos uma nova versão do conjunto de dados, sem duplicações. No entanto,
o processo de validação de integridade não acaba aqui. Ainda podem haver dados
incorretos (ou faltando) em outros formatos. A investigação segue.

In [22]:
df_clear_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29685 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             29685 non-null  object
 1   LIMIT_BAL      29685 non-null  int64 
 2   SEX            29685 non-null  int64 
 3   EDUCATION      29685 non-null  int64 
 4   MARRIAGE       29685 non-null  int64 
 5   AGE            29685 non-null  int64 
 6   PAY_1          29685 non-null  object
 7   PAY_2          29685 non-null  int64 
 8   PAY_3          29685 non-null  int64 
 9   PAY_4          29685 non-null  int64 
 10  PAY_5          29685 non-null  int64 
 11  PAY_6          29685 non-null  int64 
 12  BILL_AMT1      29685 non-null  int64 
 13  BILL_AMT2      29685 non-null  int64 
 14  BILL_AMT3      29685 non-null  int64 
 15  BILL_AMT4      29685 non-null  int64 
 16  BILL_AMT5      29685 non-null  int64 
 17  BILL_AMT6      29685 non-null  int64 
 18  PAY_AMT1       29685 non-null  

O método `.info()` revelou uma informação interessante aqui. Note que, como o
esperado, todas as nossas colunas que expressam valores numéricos são, tipicamente,
do tipo `int64`, indicando que são valores numéricos. Mas há uma excessão curiosa.

`PAY_1` é do tipo `object`. Por quê?

In [23]:
df_clear_1['PAY_1'].head()

0     2
1    -1
2     0
3     0
4    -1
Name: PAY_1, dtype: object

In [24]:
df_clear_1['PAY_1'].value_counts()

PAY_1
0                13087
-1                5047
1                 3261
Not available     3021
-2                2476
2                 2378
3                  292
4                   63
5                   23
8                   17
6                   11
7                    9
Name: count, dtype: int64

Aqui conseguimos identificar de cara a raíz do problema. Há valores `Not`
`avaiable` na coluna, e foi isso que fez o pandas ler a coluna como `object` e
não `int64`. Além disso há valores zero que não está documentado no nosso
dicionário do conjunto de dados. `PAY_1` devera ser o status do pagamento mais
recente, no qual -1 indica a auxência de atrasos, e os valores 1-9 indicam os
meses de atraso no pagamento.

Vamos precisar analisar mais a fundo

In [33]:
valid_pay_1_mask = df_clear_1['PAY_1'] != 'Not available'

valid_pay_1_mask.sum()

26664

In [34]:
df_clean_2 = df_clear_1.loc[valid_pay_1_mask, :].copy()

df_clean_2.shape

(26664, 25)